# 固定利率債建模

首先，這篇教學文章是關於如何在 QuantLib Python 中模擬固定利率債券。文章中的範例債券面值為100，半年支付6%的票息，於 2015年1月15日 發行，並設定於 2016年1月15日 到期。債券將於 2015年7月15日 和 2016年1月15日 支付票息，並在 2016年1月15日 支付面額為100的本金。文章的主要部分分為兩個部分：一部分是直接計算債券的公允價值，另一部分是使用 QuantLib 計算。

假設我們知道2015年1月15日的國庫券現貨利率。年化現貨利率為6個月0.5%，1年期點為0.7%。我們首先計算這個債券的公允價值：

In [1]:
# 用明確的數學公式計算債券的現值
3/pow(1+0.005, 0.5) + (100 + 3)/(1+0.007)

105.27653992490681

接著，我們使用 QuantLib 做同樣的計算：

In [3]:
import QuantLib as ql

# 設定今天的日期
todaysDate = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = todaysDate

# 定義現貨利率的日期和利率
spotDates = [ql.Date(15, 1, 2015), ql.Date(15, 7, 2015), ql.Date(15, 1, 2016)]
spotRates = [0.0, 0.005, 0.007]

# 定義一些需要的參數
dayCount = ql.Thirty360(ql.Thirty360.ISDA)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
interpolation = ql.Linear()
compounding = ql.Compounded
compoundingFrequency = ql.Annual

# 建立現貨利率曲線
spotCurve = ql.ZeroCurve(spotDates, spotRates, dayCount, calendar, interpolation, compounding, compoundingFrequency)
spotCurveHandle = ql.YieldTermStructureHandle(spotCurve)


至此，我們已創建了期限結構，變數相對自解釋。接下來，我們建構固定利率債券：

In [5]:
# 定義債券的發行日和到期日
issueDate = ql.Date(15, 1, 2015)
maturityDate = ql.Date(15, 1, 2016)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
bussinessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False

# 建立債券的支付日期行程
schedule = ql.Schedule(issueDate, maturityDate, tenor, calendar, bussinessConvention, bussinessConvention, dateGeneration, monthEnd)
list(schedule)  # [Date(15, 1, 12015), Date(15,7,2015), Date(15,1,2016)]

# 建立債券的票息
dayCount = ql.Thirty360(ql.Thirty360.ISDA)
couponRate = .06
coupons = [couponRate]

# 建立固定利率債券
settlementDays = 0
faceValue = 100
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)

# 建立債券引擎並設定為債券的定價引擎
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond.setPricingEngine(bondEngine)

# 最後，計算債券的現值
fixedRateBond.NPV()


105.27653992490683

## 結論

- 零息率曲線（Zero Curve）：零息率曲線是指描繪不同到期時間的零息債券收益率的曲線。在此程式碼中，我們使用ql.ZeroCurve函數來建立一條零息率曲線。我們需要提供債券的即期日期、即期利率、日計數約定、日曆以及利率的內插和複利方法。
- 固定利率債券（Fixed Rate Bond）：固定利率債券是一種債券，其利息支付是固定的，不會隨市場利率變動而變動。在此程式碼中，我們使用ql.FixedRateBond函數來建立一個固定利率債券。我們需要提供結算日數（通常為發行後的幾個工作日）、面額、付息時間表和利率。
- 債券的折現和現值計算：債券的現值是指債券未來現金流的現值之和，這通常包括債券的息票和面值。我們使用折現率（或稱為折現曲線）來將未來的現金流量折現到今天。在此程式碼中，我們使用ql.DiscountingBondEngine和ql.YieldTermStructureHandle函數來創建一個折現曲線，並使用此折現曲線來創建債券的定價引擎。最後，我們使用fixedRateBond.NPV()函數來計算債券的現值。